### Exercícios da Aula 05
---
 
Author : Halisson S. Gomides - halisson.gomides@gmail.com

Date : 09/02/2021

---

### Conjunto de dados House Sales in King County, USA
>https://www.kaggle.com/harlfoxem/housesalesprediction


### Respondendo perguntas do CEO:



In [ ]:
%config Completer.use_jedi = False 

In [ ]:
# Pesquisa uma determinada chave em um dicionário e retorna o valor correspondente, se houver
def iterdict(d:dict, chave, root=None, kp=None):
    '''
    Função para iterar recursivamente sobre um dicionario e retornar o valor da chave passada como parametro
    
    Keyword arguments:
    d -- dictionary
    chave -- name of the key to be searched
    root -- dictionary copy for internal use
    kp -- parent key for internal use
    '''
    from numpy import nan
    
    if chave in d:
        return d[chave]
    if root == None:
        droot = d.copy()
        for k,v in d.items():
            if isinstance(v, dict):
                return iterdict(v, chave, droot, k)
    else:
        for k,v in d.items():
            if isinstance(v, dict):
                del root[kp]
                return iterdict(v, chave, root, k)
        return iterdict(root, chave)
            
    return  nan


# Adiciona dados da API geopy em colunas de um dataframe e retorna o dataframe atualizado
def get_geo_attr( df, key_tuple ):
    '''
    Função para adicionar ao dataframe informações da API geopy.geocoders.
    
    Keyword arguments:
    key_tuple -- tupla com os nomes das chaves que contém as informações da API a serem adicionadas ao dataframe
    df -- dataframe que deve conter as colunas a serem populadas com os nomes das chaves key_tuple, bem como as 
    colunas "lat" e "long" com dados de latitude e longitude a serem usados como query para pesquisa na API
    '''
    
    import pandas as pd
    from geopy.geocoders import Nominatim
    from geopy.exc import GeocoderTimedOut
    from tqdm.notebook import trange
    from numpy import nan
    
    df_geo = df.copy()
    for tp in key_tuple:
        df_geo[tp] = nan
    
    geolocator = Nominatim( user_agent='geoapiExercicises' )

    for i in trange( len(df_geo) ):

        # Implementando cache caso tenha que rodar de novo o laço for. Verifica o preenchimento na coluna cujo nome 
        # é a primeteamira chave da tupla key_tuple
        if pd.isna(df_geo.iloc[i, df_geo.columns.get_loc(key_tuple[0])]) == False:
            continue

        # Montando a string de busca com os dados de 'lat' e 'long'
        query = str(df_geo.iloc[i, df_geo.columns.get_loc('lat')]) + ',' + str(df_geo.iloc[i, df_geo.columns.get_loc('long')])

        # API request
        try:
            response = geolocator.reverse(query)
        except GeocoderTimedOut:
            continue

        # Popula o dataframe com os dados da API conforme as chaves passadas na tupla key_tuple:
        for chave in key_tuple:
            df_geo.iloc[i, df_geo.columns.get_loc(chave)] = iterdict(dict(response.raw), chave)
           
    
    return df_geo


# Imprime as Dimensões do dataframe
def show_dimensions(df):
    '''
    Imprime as Dimensões do dataframe
    Keyword arguments:
    df -- dataframe
    
    '''
    print( f'Numero de linhas: {df.shape[0]}', end='\n\n' )
    print( f'Numero de colunas: {df.shape[1]}', end='\n\n' )
    return None


# Coletando o dataset
def data_collect( data_path, tipo='csv', parsedate=False ):
    '''
    Função para carregar um conjunto de dados em um DataFrame Pandas
    
    Keyword arguments:
    data_path -- caminho do arquivo que contem os dados
    tipo -- tipo do arquivo de dados: csv | xls
    '''
    
    import pandas as pd

    if tipo == 'csv':
        df_raw =  pd.read_csv( data_path, parse_dates=parsedate )
    elif tipo == 'xls':
        df_raw =  pd.read_excel( data_path, parse_dates=parsedate )
    
    show_dimensions(df_raw)
    return df_raw



# Transformacoes no dataframe
def data_transform( data ):
    
    '''
    Funcao para realizar processamentos em um DataFrame Pandas
    
    Keyword arguments:
    data -- dataframe kc_houses
    '''
    import pandas as pd
    import numpy as np
    
    # estatistica descritiva
    num_attrs = data.select_dtypes( include=['int64', 'float64'] )
    
    # tendencia central - media, mediana
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    media = pd.DataFrame( num_attrs.apply( np.mean, axis=0 ) )
    mediana = pd.DataFrame( num_attrs.apply( np.median, axis=0 ) )
    
    #  dispersao - std, min, max
    std = pd.DataFrame( num_attrs.apply( np.std, axis=0 ) )
    min_ = pd.DataFrame( num_attrs.apply( np.min, axis=0 ) )
    max_ = pd.DataFrame( num_attrs.apply( np.max, axis=0 ) )
    
    df1 = pd.concat( [max_, min_, media, mediana, std], axis=1 ).reset_index()
    df1.columns = ['attributes', 'maximo', 'minimo', 'media', 'mediana', 'std']
    print(df1)
    show_dimensions(df1)
    
    data['dormitory_type'] = data['bedrooms'].apply(lambda n: 'house' if n > 2 else ('apartment' if n == 2 else 'studio')) 
    
    data['level'] = data['price'].apply(lambda preco:
                                            'nivel_0' if 0 <= preco < 321950 else(
                                                'nivel_1' if 321950 <= preco < 450000 else(
                                                    'nivel_2' if 450000 <= preco < 645000 else 'nivel_3'
                                                )
                                            )
                                        )
    
    cols = ('road', 'house_number')
    df = data.head(20)
    df2 = get_geo_attr(df, cols)
    show_dimensions(df2)
    
    return df2
    

# Vizualização do mapa
def data_load( data ):
    
    #------------------------------
    # Load
    #------------------------------
    lst_columns = ['id', 'lat', 'long', 'price', 'level']
    df_houses = data[lst_columns].copy()
    
    mapa = px.scatter_mapbox( df_houses, lat='lat', lon='long',  
                         hover_name='id',
                         hover_data=['price'],
                         color_continuous_scale= px.colors.diverging.Portland,
                         color='level',
                         size='price',
                         size_max=15,
                         zoom= 9)
    mapa.update_layout(mapbox_style='open-street-map')
    mapa.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
    mapa.show()
    return None



In [ ]:
if __name__ == '__main__':
    #  ETL
    #  Collect
    data_raw = data_collect('dados/originais/kc_house_data.csv.zip', parsedate=['date'])
    
    #  Transform
    data_processing = data_transform( data_raw )
    
    #  Load
    data_load( data_processing )
    

In [1]:
dict_teste = {'teste': 'valor1', 
              'A': {'a1': 'vn1', 'a2': 'vn2', 'a3': {'a31': 'vnn31'}}, 
              'B': 'valor2', 
              'C': {'c1': 'vn3', 
                    'c2': {'c21': 'vnn1', 'c22': 'vnn2', 'c23': {'c231': 'vnnn3'}},
                    'c3': 'vn4'},
              'D': 'valor3'}

In [ ]:
iterdict(dict_teste, 'c231')

In [ ]:
'''
1 - {'a1': 'vn1', 'a2': 'vn2', 'a3': {'a31': 'vnn31'}}, root=original, kp=A
2 - {'a31': 'vnn31'}, root=original, kp=a3
{'c21': 'vnn1', 'c22': 'vnn2', 'c23': {'c231': 'vnnn3'}, 
'''

In [ ]:
def iterdict(d:dict, chave, root=None, kp=None):
    if chave in d:
        return d[chave]
    if root == None:
        droot = d.copy()
        for k,v in d.items():
            if isinstance(v, dict):
                return iterdict(v, chave, droot, k)
    else:
        for k,v in d.items():
            if isinstance(v, dict):                
                return iterdict(v, chave, root, k)
        del root[kp]
        return iterdict(root, chave)
            
    return  nan